In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.graphics import tsaplots
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults, ARMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy import signal
from sklearn.metrics import mean_squared_error

# from .ARIMA_functions import get_ARIMA_model
# , plot_ARIMA_model, plot_ARIMA_resids, get_ARIMA_forecast

# ,\
# plot_data_plus_ARIMA_predictions, plot_data_plus_ARIMA_predictions, test_rolling_ARIMA_forecast,\
# plot_rolling_ARIMA_forecast, get_predictions_df_and_plot_rolling_ARIMA_forecast

In [ ]:
# from ./capstoneARIMA_functions import get_ARIMA_model


### import data

In [ ]:
duration_train_df = pd.read_csv('hours_training_data.csv', parse_dates=True, index_col='AppointmentDate')

In [ ]:
duration_test_df = pd.read_csv('hours_test_data.csv', parse_dates=True, index_col='AppointmentDate')

In [ ]:
all_data_df = pd.concat([duration_train_df, duration_test_df], axis=0)

In [ ]:
all_data_df.isnull().sum()

### Separate training and test df by specialty, then downsample to get hours/week

#### training data

In [ ]:
doctors = duration_train_df[duration_train_df['Specialty'] == 'doctor']
therapists = duration_train_df[duration_train_df['Specialty'] == 'therapist']
RN_PA = duration_train_df[duration_train_df['Specialty'] == 'RN/PA']

In [ ]:
# Group by hours per day
doc_duration = doctors.groupby(doctors.index.date)['Hours_Spent'].sum()
RN_PA_duration = RN_PA.groupby(RN_PA.index.date)['Hours_Spent'].sum()
therapist_duration = therapists.groupby(therapists.index.date)['Hours_Spent'].sum()

In [ ]:
def index_to_datetime(series):
    "Converts series object indext to datetime"
    series.index = pd.to_datetime(series.index, errors='coerce')

In [ ]:
training_duration = [doc_duration, RN_PA_duration, therapist_duration]

In [ ]:
# convert training data index to datetime
for item in training_duration:
    index_to_datetime(item)

In [ ]:
# downsample from daily to weekly data, filling missing data w/ the mean
def downsample_data_week(data, fill_method='bfill'):
    downsampled = data.resample(rule='W').mean() 
    downsampled.fillna(method=fill_method, inplace=True)
    return downsampled

In [ ]:
dr_train_data = downsample_data_week(doc_duration, fill_method='bfill')
RN_PA_train_data = downsample_data_week(RN_PA_duration, fill_method='bfill')
therapist_train_data = downsample_data_week(therapist_duration, fill_method='bfill')

#### all data

In [ ]:
all_doctors = all_data_df[all_data_df['Specialty'] == 'doctor']
all_therapists = all_data_df[all_data_df['Specialty'] == 'therapist']
all_RN_PA = all_data_df[all_data_df['Specialty'] == 'RN/PA']

In [ ]:
# Group by hours per day
all_doc_duration = all_doctors.groupby(all_doctors.index.date)['Hours_Spent'].sum()
all_RN_PA_duration = all_RN_PA.groupby(all_RN_PA.index.date)['Hours_Spent'].sum()
all_therapist_duration = all_therapists.groupby(all_therapists.index.date)['Hours_Spent'].sum()

In [ ]:
all_data = [all_doc_duration, all_RN_PA_duration, all_therapist_duration]

In [ ]:
for item in all_data:
    index_to_datetime(item)

In [ ]:
all_dr = downsample_data_week(all_doc_duration, fill_method='bfill')
all_RN_PA = downsample_data_week(all_RN_PA_duration, fill_method='bfill')
all_therapist = downsample_data_week(all_therapist_duration, fill_method='bfill')

In [ ]:
all_data_sets = [all_dr, all_RN_PA, all_therapist]

#### test data

In [ ]:
dr_test_data = duration_test_df[duration_test_df['Specialty'] == 'doctor']
RN_PA_test_data = duration_test_df[duration_test_df['Specialty'] == 'RN/PA']
therapist_test_data = duration_test_df[duration_test_df['Specialty'] == 'therapist']

In [ ]:
# dr_test_data = duration_test_data[duration_test_data['Specialty'] == 'doctor']
# RN_PA_test_data = duration_test_data[duration_test_data['Specialty'] == 'RN/PA']
# therapist_test_data = duration_test_data[duration_test_data['Specialty'] == 'therapist']

In [ ]:
dr_test_data = dr_test_data.groupby(dr_test_data.index.date)['Hours_Spent'].sum()
RN_PA_test_data = RN_PA_test_data.groupby(RN_PA_test_data.index.date)['Hours_Spent'].sum()
therapist_test_data = therapist_test_data.groupby(therapist_test_data.index.date)['Hours_Spent'].sum()

In [ ]:
test_data = [dr_test_data, RN_PA_test_data, therapist_test_data]

In [ ]:
# convert test data index to datetime
for item in test_data:
    index_to_datetime(item)

In [ ]:
# def get_test_data(data):
#     data = data['AppointmentDuration']
#     data = data.resample(rule='W').mean()
#     data.fillna(method='bfill', inplace=True)
#     return data

In [ ]:
dr_test_set = downsample_data_week(dr_test_data)
RN_PA_test_set = downsample_data_week(RN_PA_test_data)
therapist_test_set = downsample_data_week(therapist_test_data)

In [ ]:
def plot_series(series, xlabel, ylabel, plot_name):
    "Plots simple time series from Pandas Series"
    ax = series.plot(figsize=(8,3), linewidth = 3, fontsize=10, grid=True, rot=30)
    ax.set_title(plot_name, fontsize=18)
    ax.set_xlabel(xlabel, fontsize=15)
    ax.set_ylabel(ylabel, fontsize=15)
    plt.show()

In [ ]:
def plot_series_and_differences(series, ax, num_diff, title):
    "Plot raw data and specified number of differences"
    ax[0].plot(series.index, series)
    ax[0].set_title('Raw series: {}'.format(title))
    for i in range(1, num_diff+1):
        diff = series.diff(i)
        ax[i].plot(series.index, diff)
        ax[i].set_title('Difference # {}'.format(str(i)))   

In [ ]:
plot_series(series=dr_train_data, xlabel='', ylabel='', plot_name='Doctors')

In [ ]:
# plot raw data and first and second differences for doctors
fig, axes = plt.subplots(3, figsize=(10, 8))
plot_series_and_differences(series=dr_train_data, ax=axes, num_diff=2, title='Doctors')
fig.tight_layout()

In [ ]:
def run_augmented_Dickey_Fuller_test(series, num_diffs=None):
    test = sm.tsa.stattools.adfuller(series)
    if test[1] >= 0.05:
        print('The p-value for the series is: {p}, which is not significant'.format(p=test[1]))
    else:
        print('The p-value for the series is: {p}, which is significant'.format(p=test[1]))  
    if num_diffs:
        for i in range(1, num_diffs +1):
            test = sm.tsa.stattools.adfuller(series.diff(i)[i:])
            if test[1] >= 0.05:
                print('The p-value for difference {diff} is: {p}, which is not significant'.format(diff=str(i), p=test[1]))
            else:
                print('The p-value for difference {diff} is: {p}, which is significant'.format(diff=str(i), p=test[1]))   

In [ ]:
# test for stationarity of doctors data, 1st and 2nd diff
run_augmented_Dickey_Fuller_test(doc_duration, num_diffs=2)

In [ ]:
# plot raw data and first and second differences for RN/PA
fig, axes = plt.subplots(3, figsize=(10, 8))
plot_series_and_differences(series=RN_PA_train_data, ax=axes, num_diff=2, title='RN/PA')
fig.tight_layout()

In [ ]:
# test for stationarity of RN/PA data, 1st and 2nd diff
run_augmented_Dickey_Fuller_test(RN_PA_duration, num_diffs=2)

In [ ]:
# plot raw data and first and second differences for therapists
fig, axes = plt.subplots(3, figsize=(10, 8))
plot_series_and_differences(series=therapist_train_data, ax=axes, num_diff=2, title='Therapists')
fig.tight_layout()

In [ ]:
# test for stationarity of therapist data, 1st and 2nd diff
run_augmented_Dickey_Fuller_test(therapist_duration, num_diffs=2)

In [ ]:
def plot_autocorrelation(series, params, lags, alpha, title):
    plt.rcParams.update(params)
    acf_plot = tsaplots.plot_acf(series, lags=lags, alpha=alpha)
    plt.title(title)
    plt.xlabel('Number of Lags')
    plt.show()

def plot_partial_autocorrelation(series, params, lags, alpha, title):
    plt.rcParams.update(params)
    acf_plot = tsaplots.plot_pacf(series, lags=lags, alpha=alpha)
    plt.xlabel('Number of Lags')
    plt.title(title)
    plt.show()

#### plot decomposition for all categories weekly data

In [ ]:
def plot_decomposition(series, params, freq, title=''):
    "Plots observed, trend, seasonal, residual"
    plt.rcParams.update(params)
    decomp = sm.tsa.seasonal_decompose(series, freq=freq)
    fig = decomp.plot()
    plt.title(title)
    plt.show()

In [ ]:
# series_list = training_datasets
# for i in range(3):
#     params = {'figure.figsize': [12, 8], 'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
#     axes[i] = plot_decomposition(series=series_list[i], params=params, freq=31)

In [ ]:
# doctors
params = {'figure.figsize': [8, 8],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_decomposition(dr_train_data, params=params, freq=31, title='Doctors Decomposition')

In [ ]:
# RN/PA
params = {'figure.figsize': [8,8],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_decomposition(RN_PA_train_data, params=params, freq=31, title='RN/PA Decomposition')

In [ ]:
# plot_series(weekly_doc_dur, xlabel='Date', ylabel='Hours', plot_name='Doctor Hours per Week')

In [ ]:
# therapists
params = {'figure.figsize': [8,8],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_decomposition(therapist_train_data, params=params, freq=31, title='Therapist Decomposition')